"""
---------------------------------------------------------
Data Preprocessing Script (Step 1): Feature Engineering and Filtering
---------------------------------------------------------

This script performs the first stage of data processing for the artwork price prediction pipeline. 
It reads and merges auction and artist data, creates new variables, standardizes categorical values, 
and filters the dataset in preparation for imputation and modeling.

Main Tasks:
-----------
1. **Load Data**:
   - Loads auction results and artist metadata from pickle files.
   - Loads and normalizes U.S. CPI data for inflation adjustment.

2. **Feature Engineering**:
   - Extracts and cleans artist country, year of birth/death.
   - Derives "Birth Period" and "Alive Status at Time of Sale".
   - Adds "Sale Year" and maps CPI based on year of sale.

3. **Category Processing**:
   - Groups infrequent countries into "Others" (top 40 retained).
   - Groups infrequent auction houses into "Others" (top 100 retained).

4. **Filtering**:
   - Removes rows where final sale price or sale date is missing.
   - Reports missing data summary before and after filtering.

Inputs:
-------
- 'Cleaned_Data/auction_results_cleaned.pickle' : list of auction results 
- 'Cleaned_Data/artists_details.pickle'         : list of artist metadata (Name, Country, Biography)
- 'cpi_us.csv'                                   : inflation index data

Output:
-------
- `df` : a cleaned, CPI-adjusted DataFrame with engineered features
         ready for imputation of paint and material fields in the next step.

Next Step:
----------
- Imputation of 'Material' and 'Paint' columns.

"""

In [1]:
# Libraries
import pandas as pd
import numpy as np
import re
import pickle

In [ ]:
import os
os.chdir('Art-Valuation-in-Auction')

In [3]:
# Reading Pickle File Function
def read_pickle_to_dataframe(filepath):
    try:
        with open(filepath, "rb") as f:
            data = []
            while True:
                try:
                    obj = pickle.load(f)
                    if isinstance(obj, list):
                        data.extend(obj)
                    else:
                        data.append(obj)
                except EOFError:
                    break
        print(f"Loaded {len(data)} records.")
        df = pd.DataFrame(data)
        print("Columns:", df.columns.tolist())
        return df
    except Exception as e:
        print(f"Error reading pickle file: {e}")
        return pd.DataFrame()

In [ ]:
# -------------------------------
# Load Auction and Artist Data
# -------------------------------
df = read_pickle_to_dataframe("Datasets/auction_results_cleaned.pickle")

artists_details = read_pickle_to_dataframe('Datasets/artists_details.pickle')
artists_details_df = pd.DataFrame(artists_details)

In [ ]:
# -------------------------------
# Extract Country and Date of Birth from Artist Metadata
# -------------------------------
def clean_country_year(country_year):
    
    """
     Parses 'country, YYYY–YYYY' string into components:
        - Country
        - Year of Birth (yob)
        - Year of Death (yod)
    """

    country, year = '', ''
    yob, yod = np.nan, np.nan

    if pd.isnull(country_year):
        return pd.Series([country, yob, yod])

    country_year = re.split(',', country_year.strip(), maxsplit=1)

    if len(country_year) == 2:
        country, year = country_year
    elif len(country_year) == 1:
        if re.search(r'\d{2,}', country_year[0]):
            year = country_year[0]
        else:
            country = country_year[0]

    if '-' in year or '–' in year:
        year = re.findall(r'(?:^|\D)(\d+)\s*[–\-]\s*(\d+)(?:$|\D)', year)
        if len(year) == 1:
            year = year[0]
            if len(year) == 2:
                yob, yod = year
                yob = int(yob) if len(yob) == 4 else np.nan
                yod = int(yod) if len(yod) == 4 else np.nan

    elif year != '':
        yob = re.findall(r'(?:^|\D)(\d{4})(?:\D|$)', year)
        yob = int(yob[0]) if len(yob) == 1 else np.nan

    return pd.Series([country.strip(), yob, yod])


artists_details_df[['Country', 'Year of Birth', 'Year of Death']] = (
    artists_details_df['artist_country_year'].apply(clean_country_year)
)
artists_details_df['Country'] = artists_details_df['Country'].replace('', 'Unknown')

In [ ]:
# -------------------------------
# Log Area
# -------------------------------
df['Log Area'] = np.log(df['Area'].replace(0, np.nan))

In [ ]:
# -------------------------------
# Derive Birth Period from Year of Birth
# -------------------------------

# Define Periods Manually
periods_year = [0, 1400, 1600, 1750, 1850, 1950, 1970, 2025]
labels = [f'{periods_year[i]}-{periods_year[i+1]}' for i in range(len(periods_year) - 1)]


for i in range(len(periods_year) - 1):
    mask = (artists_details_df['Year of Birth'] > periods_year[i]) & \
           (artists_details_df['Year of Birth'] <= periods_year[i + 1])
    artists_details_df.loc[mask, 'Birth Period'] = labels[i]

# Fill Missing Periods as 'Unknown'
artists_details_df['Birth Period'] = artists_details_df['Birth Period'].fillna('Unknown')


# Birth Period as Ordinal Category
birth_period_order = labels + ['Unknown']
birth_period_map = {label: i for i, label in enumerate(birth_period_order)}
artists_details_df['Birth Period Ordinal'] = artists_details_df['Birth Period'].map(birth_period_map)


In [ ]:
# ----------------------------------------
# Merge Artist Metadata into Auction Data
# ----------------------------------------

df = df.merge(
    artists_details_df[['artist_id', 'Country', 'Year of Birth', 'Year of Death', 'Birth Period', 'Birth Period Ordinal']],
    how='left',
    left_on='Artist ID',
    right_on='artist_id'
)
df.drop(columns=['artist_id'], inplace=True)

# Extract year from sale date for downstream use
df['Sale Year'] = df['Sale Date Cleaned'].dt.year

In [ ]:
# ----------------------------------------
# US Consumer Price Index (CPI) Adjustment
# ----------------------------------------

# Load CPI data, use data from 1985 onwards
# Our auctions range from 1985 (earliest) to 2025 (latest)
cpi_us = pd.read_csv('cpi_us.csv', header=0)
cpi_us.columns = ['YEAR', 'CPI']
cpi_us = cpi_us[cpi_us['YEAR'] >= 1985].copy()
cpi_us['CPI'] = cpi_us['CPI'].replace('–', np.nan).astype(float)

# Normalize CPI to 1996 baseline
cpi_us_1996 = cpi_us.loc[cpi_us['YEAR'] == 1996, 'CPI'].iloc[0]
cpi_us['CPI'] = cpi_us['CPI'] / cpi_us_1996

# Project 2025 CPI using estimated 2.6% inflation over 2024
cpi_us_2024 = cpi_us.loc[cpi_us['YEAR'] == 2024, 'CPI'].iloc[0]
cpi_us.loc[cpi_us['YEAR'] == 2025, 'CPI'] = cpi_us_2024 * 1.026

# Map CPI to artworks based on sale year
cpi_map = cpi_us.set_index('YEAR')['CPI'].to_dict()
df['CPI_US'] = df['Sale Date Cleaned'].dt.year.map(cpi_map)

# Identify any years in the data that are missing CPI info
missing_years = set(df['Sale Date Cleaned'].dt.year.unique()) - set(cpi_map)
if missing_years:
    print("Missing CPI for years:", missing_years)

In [ ]:
# ----------------------------------------
# Determine Whether Artist Was Alive at Time of Sale
# ----------------------------------------

def assign_alive_status(row):
    """
    Assigns alive/dead status at time of sale:
        - 1: Alive
        - 0: Dead
        - 'Unknown': if unknown
    
    Fallback estimation is used when year of death is missing:
        - If artist was <50 years old → probably alive
        - If artist was >100 years old → probably dead
        - Otherwise → status remains 'Unknown'
    """
    if pd.notnull(row['Year of Death']):
        if row['Sale Year'] < row['Year of Death']:
            return 1  # Alive
        elif row['Sale Year'] > row['Year of Death']:
            return 0  # Dead
    else:
        # Estimate based on age at time of sale
        if row['Sale Year'] < row['Year of Birth'] + 50:
            return 1  # Probably alive
        elif row['Sale Year'] > row['Year of Birth'] + 100:
            return 0  # Probably dead
    return 'Unknown'  # Can't determine

# Apply the logic row-wise
df['Alive Status'] = df.apply(assign_alive_status, axis=1)

In [ ]:
# ----------------------------------------
# Group Countries by Frequency
# ----------------------------------------

# Count frequency of each country
country_summary = df['Country'].value_counts().reset_index()
country_summary.columns = ['Country', 'count']
country_summary['Rank'] = range(len(country_summary))
country_summary['Cover'] = country_summary['count'].cumsum() / len(df)

# Keep only the top 40 most frequent countries
# All others will be grouped under 'Others' to reduce category cardinality
top_countries = country_summary.head(40)['Country'].tolist()
df['Country'] = df['Country'].where(df['Country'].isin(top_countries), other='Others')


In [ ]:
# ----------------------------------------
# Missing Value Summary (Before Filtering)
# ----------------------------------------

# Count and percentage of missing values per column
missing_counts = df.isnull().sum()
missing_percent = df.isnull().mean() * 100

missing_summary = pd.DataFrame({
    'Missing Values': missing_counts,
    'Percent Missing (%)': missing_percent.round(2)
})

print("Missing values before filtering:")
print(missing_summary)

In [ ]:
# ----------------------------------------
# Remove Rows with Missing Target or Date
# ----------------------------------------

df = df[df['Price Sold USD'].notnull()]         # Drop if final sale price is missing
df = df[df['Sale Date Cleaned'].notnull()]      # Drop if sale date is missing


In [ ]:
# ----------------------------------------
# Missing Value Summary (After Filtering)
# ----------------------------------------

# Recompute missing value summary after filtering
missing_counts = df.isnull().sum()
missing_percent = df.isnull().mean() * 100

missing_summary = pd.DataFrame({
    'Missing Values': missing_counts,
    'Percent Missing (%)': missing_percent.round(2)
})

print("Missing values after filtering:")
print(missing_summary)

In [ ]:
# ----------------------------------------
# Final Data Summary: Columns, Unique Values, Missing Values
# ----------------------------------------

summary_df = pd.DataFrame({
    'Column Name': df.columns,
    'Unique Values': [df[col].nunique(dropna=True) for col in df.columns],
    'Missing Values': df.isnull().sum().values
})

print("Final Column Summary:")
print(summary_df.sort_values(by='Missing Values', ascending=False).reset_index(drop=True))

In [ ]:
# ----------------------------------------
# Save Final DataFrame for Next Processing Step
# ----------------------------------------

output_path = "Datasets/DataProcessing_Step1_df.pkl"
df.to_pickle(output_path)